In [1]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import pickle
import random

Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Europe + NorthAmerica + SouthAmerica + Oceania
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

w_length = 200

### Consolidate and pickle country files

In [2]:
for country in countriesOfInterest:
    if len(glob.glob("Raw Track Data\\" + country + ".p")) == 0 :
        hold = pd.DataFrame()
    else:
        hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    print(country, hold.shape)
    countryTracks = glob.glob("Raw Track Data\\" + country + "*.csv")
    print(country, len(countryTracks))
    for file in countryTracks:
        new = pd.read_csv(file)
        new["Country"] = file[15:17]
        new["Year"] = file[18:22]
        new["Playlist"] = file[23:-4]
        hold = hold.append(new)
        pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )
        os.remove(file)
    hold = hold.drop_duplicates(["track_id", "start"])
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

DZ (22029, 34)
DZ 17
EG (270561, 34)
EG 16
MA (27489, 34)
MA 17
ZA (1665447, 34)
ZA 11
TN (63078, 34)
TN 17
BH (213647, 34)
BH 16
HK (614936, 34)
HK 12
IN (381200, 34)
IN 12
ID (1440470, 34)
ID 7
IL (1047848, 34)
IL 11
JP (1451311, 34)
JP 7
JO (50762, 34)
JO 16
KW (34610, 34)
KW 16
LB (3063, 34)
LB 16
MY (419460, 34)
MY 12
OM (1845, 34)
OM 15
PS (11523, 34)
PS 11
PH (754250, 34)
PH 0
QA (1643, 34)
QA 0
SA (0, 34)
SA 0
SG (421199, 34)
SG 0
TW (2083768, 34)
TW 0
TH (1251782, 34)
TH 0
AE (559535, 34)
AE 0
VN (1261207, 34)
VN 0
AD (24354, 34)
AD 0
AT (477748, 34)
AT 0
BE (686798, 34)
BE 0
BG (7218, 34)
BG 0
CY (159637, 34)
CY 0
CZ (82775, 34)
CZ 0
DK (2435298, 34)
DK 0
EE (2386, 34)
EE 0
FI (1763553, 34)
FI 0
FR (1647130, 34)
FR 0
DE (714391, 34)
DE 0
GR (0, 34)
GR 0
HU (6322, 34)
HU 0
IS (271525, 34)
IS 0
IE (1519446, 34)
IE 0
IT (859501, 34)
IT 9
LV (1966, 34)
LV 12
LI (0, 34)
LI 12
LT (0, 34)
LT 12
LU (1402, 34)
LU 12
MT (0, 34)
MT 12
MC (3856, 34)
MC 12
NL (1196994, 34)
NL 13
NO (12215

In [3]:
hold

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,p1,p2,p3,p4,...,t7,t8,t9,t10,t11,t12,track_id,Country,Year,Playlist
681,0.00000,0.49914,0.000,-60.000,0.00000,-60.000,0.019,0.040,0.045,0.024,...,14.833,5.359,-27.228,0.973,-10.640,-7.228,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
682,0.49914,0.22644,1.000,-60.000,0.03370,-10.164,0.646,0.420,0.522,0.560,...,-20.233,-14.167,-6.882,34.476,41.585,7.220,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
683,0.72558,0.12181,0.381,-31.803,0.00782,-26.013,0.940,0.858,0.439,0.460,...,16.783,-18.108,13.722,32.195,-23.695,-3.539,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
684,0.84739,0.60340,0.952,-26.892,0.18692,-14.485,0.073,0.149,0.097,0.237,...,64.849,-42.428,21.399,50.362,-43.406,-18.990,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
685,1.45079,0.20980,0.775,-21.574,0.04262,-13.192,0.296,0.377,0.302,0.450,...,38.661,-2.748,52.698,13.987,-17.991,-5.808,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35115,244.23868,0.13347,0.320,-39.519,0.05188,-35.125,0.043,0.067,1.000,0.368,...,-32.935,-11.326,-10.070,5.579,-13.756,-25.416,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,NZ,2012,37i9dQZF1DXc78A1OIheFs
35116,244.37215,0.46481,0.389,-38.014,0.05181,-34.132,0.052,0.074,1.000,0.425,...,-49.382,32.281,-13.598,-3.731,-46.181,-32.103,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,NZ,2012,37i9dQZF1DXc78A1OIheFs
35117,244.83696,0.30113,0.416,-49.409,0.12787,-44.678,0.033,0.033,1.000,0.165,...,-21.475,0.483,-30.965,13.957,-7.462,-43.662,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,NZ,2012,37i9dQZF1DXc78A1OIheFs
35118,245.13810,0.44794,0.471,-47.469,0.04683,-43.302,0.047,0.083,1.000,0.318,...,-46.128,60.577,-22.883,-4.741,-32.316,-44.355,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,NZ,2012,37i9dQZF1DXc78A1OIheFs


### Remove non-unique tracks

In [4]:
allTracks = pd.DataFrame()
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[["Country", "Year", "track_id", "Playlist"]]
    allTracks = allTracks.append(hold)
allTracks

,Country,Year,track_id,Playlist
19557,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19558,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19559,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19560,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19561,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
...,...,...,...,...
35115,NZ,2012,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,37i9dQZF1DXc78A1OIheFs
35116,NZ,2012,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,37i9dQZF1DXc78A1OIheFs
35117,NZ,2012,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,37i9dQZF1DXc78A1OIheFs
35118,NZ,2012,spotify:track:6T5LmTL8mxFkvWh0Xj4tOl,37i9dQZF1DXc78A1OIheFs


In [5]:
hold = allTracks.groupby("track_id").nunique()
print(hold.shape[0], "Unique tracks per country")
keep = hold[hold.Country==1].index
allTracks = allTracks[allTracks.track_id.isin(keep)]
print(len(keep), "Unique tracks across countries")

61778 Unique tracks per country
56392 Unique tracks across countries


In [6]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[hold.track_id.isin(keep)]
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

### Create training, testing, and validation datasets

In [7]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    test = hold.loc[hold.track_id.isin(testTracks)]
    hold = hold.loc[~hold.track_id.isin(testTracks)]
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    val = hold.loc[hold.track_id.isin(testTracks)]
    train = hold.loc[~hold.track_id.isin(testTracks)]
    pickle.dump( train, open( "Raw Track Data\\" + country + "_train.p", "wb" ) )
    pickle.dump( test, open( "Raw Track Data\\" + country + "_test.p", "wb" ) )
    pickle.dump( val, open( "Raw Track Data\\" + country + "_val.p", "wb" ) )

In [8]:
countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']
for country in countriesOfInterest:
    train = pickle.load( open( "Raw Track Data\\" + country + "_train.p", "rb" ) )
    test = pickle.load( open( "Raw Track Data\\" + country + "_test.p", "rb" ) )
    val = pickle.load( open( "Raw Track Data\\" + country + "_val.p", "rb" ) )
    print(country, "train", len(pd.unique(train.track_id)), 
          "\t test",len(pd.unique(test.track_id)) , 
          "\t val", len(pd.unique(val.track_id)))

HK train 707 	 test 220 	 val 176
JP train 1191 	 test 372 	 val 297
ZA train 1229 	 test 383 	 val 307
TN train 42 	 test 12 	 val 10
TR train 1175 	 test 366 	 val 293
GB train 631 	 test 196 	 val 157
MX train 522 	 test 162 	 val 130
US train 865 	 test 270 	 val 216
CO train 804 	 test 251 	 val 201
EC train 237 	 test 74 	 val 59
AU train 637 	 test 198 	 val 159
NZ train 406 	 test 126 	 val 101
